In [1]:
import wandb
import pandas as pd
import numpy as np
import seaborn as sns
from pydantic import BaseModel
import matplotlib.pyplot as plt

In [2]:
class RunData(BaseModel):
    run_path: str
    train_accuracy_key: str = "train_accuracy"

In [3]:
RUNS: dict[str, RunData] = {
    "InceptionV3-No-Preprocessing-100-Epoch": RunData(
        run_path="silvan-wiedmer-fhgr/VisionTransformer/uxlv6ohn"
    ),
    "DeiT-No-Preprocessing-100-Epoch": RunData(
        run_path="silvan-wiedmer-fhgr/VisionTransformer/tudgtvng"
    ),
    "Hybrid-Preprocessing-100-Epoch": RunData(
        run_path="silvan-wiedmer-fhgr/VisionTransformer/arj9940n"
    ),
}

In [4]:
RUNS: dict[str, RunData] = {
    "CNN-Preprocessing": RunData(run_path="silvan-wiedmer-fhgr/VisionTransformer/54zn25vw", train_accuracy_key="epoch/accuracy"),
    "CNN-No-Preprocessing": RunData(run_path="silvan-wiedmer-fhgr/VisionTransformer/qjyk03li", train_accuracy_key="epoch/accuracy"),
    "InceptionV3-Preprocessing": RunData(run_path="silvan-wiedmer-fhgr/VisionTransformer/7jsq6j0z", train_accuracy_key="epoch/accuracy"),
    "InceptionV3-No-Preprocessing": RunData(run_path="silvan-wiedmer-fhgr/VisionTransformer/fudrwisc", train_accuracy_key="epoch/accuracy"),
    "ViT-Preprocessing": RunData(run_path="silvan-wiedmer-fhgr/VisionTransformer/o0s7du2i"),
    "ViT-No-Preprocessing": RunData(run_path="silvan-wiedmer-fhgr/VisionTransformer/kd0mr7el"),
    "ViT-Lite-Preprocessing": RunData(run_path="silvan-wiedmer-fhgr/VisionTransformer/5622jway"),
    "ViT-Lite-No-Preprocessing": RunData(run_path="silvan-wiedmer-fhgr/VisionTransformer/2i5780q1"),
    "DeiT-Preprocessing": RunData(run_path="silvan-wiedmer-fhgr/VisionTransformer/uf8vz397"),
    "DeiT-No-Preprocessing": RunData(run_path="silvan-wiedmer-fhgr/VisionTransformer/q3kr64sx"),
    "Hybrid-Preprocessing": RunData(run_path="silvan-wiedmer-fhgr/VisionTransformer/mdw8ucu3"),
    "Hybrid-No-Preprocessing": RunData(run_path="silvan-wiedmer-fhgr/VisionTransformer/tttuvx7n")
}

In [5]:
api = wandb.Api()

In [6]:
def get_runtime_seconds(df: pd.DataFrame, key: str = "_runtime") -> np.float64:
    return df[key].values[-1]

In [7]:
def ws_to_kwh(ws: np.float64) -> np.float64:
    return ws / (3600 * 1000)

In [8]:
def calculate_sustainable_accuracy_metric(accuracy: float, electricity_ws: float, alpha: float = 5) -> float:
    return accuracy**alpha / np.log10(electricity_ws)

In [9]:
model_performance = {
    "Model": [],
    "EnergyConsumptionkWh": [],
    "EnergyConsumptionWs": [],
    "TestAccuracy": [],
    "TestF1": [],
    "Epochs": [],
    "Runtime": []
}

In [ ]:
for run_name, run_data in RUNS.items():
    print(run_name, run_data)
    run = api.run(run_data.run_path)
    df_system_metrics = run.history(stream="system")

    df_power_usage = pd.DataFrame({"power": df_system_metrics["system.gpu.0.powerWatts"], "runtime": df_system_metrics["_runtime"]})
    df_power_usage.dropna(inplace=True)

    model_performance["Model"].append(run_name)
    model_performance["EnergyConsumptionkWh"].append(ws_to_kwh(np.trapezoid(df_power_usage["power"], df_power_usage["runtime"])))
    model_performance["EnergyConsumptionWs"].append(np.trapezoid(df_power_usage["power"], df_power_usage["runtime"]))
    model_performance["TestAccuracy"].append(run.summary.get("test_acc") * 100)
    model_performance["TestF1"].append(run.summary.get("test_f1") * 100)
    model_performance["Epochs"].append(len(run.history()[run_data.train_accuracy_key].dropna()))
    model_performance["Runtime"].append(run.summary.get("_runtime"))

In [ ]:
model_performance

# Export Model Performance

In [ ]:
df_model_performance = pd.DataFrame(model_performance)
df_model_performance

In [13]:
df_model_performance[["Model", "TestAccuracy", "TestF1"]].sort_values(
    by="TestAccuracy", ascending=False
).rename(columns={"TestAccuracy": "Accuracy", "TestF1":"F1-Score"}).to_latex(
    index=False,
    formatters={"name": str.upper},
    float_format="{:.4f}".format,
    buf="final_model_metrics.tex",
)

In [14]:
df_model_performance["AccuracyPerkWh"] = df_model_performance["TestAccuracy"] / df_model_performance["EnergyConsumptionkWh"]

In [15]:
df_model_performance["SAM"] = calculate_sustainable_accuracy_metric(df_model_performance["TestAccuracy"], df_model_performance["EnergyConsumptionWs"], alpha=5)

In [16]:
df_model_performance["Duration/Epoch"] = df_model_performance["Runtime"] / df_model_performance["Epochs"]

In [ ]:
df_model_performance

In [18]:
from datetime import timedelta
def timedelta_to_string(delta: float) -> str:
    total_seconds = timedelta(seconds=delta).total_seconds()
    days = total_seconds // (24 * 3600)
    hours = (total_seconds % (24 * 3600)) // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60

    time_str = []

    if days > 0:
        time_str.append(f"{int(days)} days")
    if hours > 0:
        time_str.append(f"{int(hours)} hours")
    if minutes > 0:
        time_str.append(f"{int(minutes)} min")
    if seconds > 0:
        time_str.append(f"{int(seconds)} s")

    return " ".join(time_str)

In [ ]:
timedelta_to_string(85.65)

In [20]:
df_model_performance["Runtime"] = df_model_performance["Runtime"].apply(lambda x: timedelta_to_string(x))
df_model_performance["Duration/Epoch"] = df_model_performance["Duration/Epoch"].apply(lambda x: timedelta_to_string(x))

In [ ]:
df_model_performance[["Model", "Epochs", "Runtime", "Duration/Epoch"]].to_latex(
    index=False,
    formatters={"name": str.upper},
    float_format="{:.4f}".format,
    buf="final_model_runtime.tex",
)

In [ ]:
ax = sns.barplot(df_model_performance.sort_values(["SAM"]), x="SAM", y="Model", orient="y")
ax.set_title("SAM with alpha = 1")

- Log 10 of Energy Consumption in Ws

In [67]:
df_model_performance["EnergyConsumptionWs"] = np.log10(df_model_performance["EnergyConsumptionWs"])

In [ ]:
ax = sns.scatterplot(df_model_performance, x="EnergyConsumptionWs", y="TestAccuracy", hue="Model")
ax.set_title("Test Accuracy vs Energy Consumption")
ax.set(xlabel="Energy Consumption [Ws]", ylabel="Accuracy [%]")
ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2), ncol=3)
plt.savefig("../images/AccuracyPerEnergy.png", dpi=600, bbox_inches="tight")

In [ ]:
ax = sns.scatterplot(df_model_performance, x="EnergyConsumptionWs", y="Epochs", hue="Model")
ax.set_title("Epochs vs Energy Consumption")
ax.set(xlabel="Energy Consumption [Ws]", ylabel="Epochs [1]")
ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2), ncol=3)

In [ ]:
ax = sns.scatterplot(df_model_performance, x="Epochs", y="TestAccuracy", hue="Model")
ax.set_title("Test Accuracy vs Epochs")
ax.set(xlabel="Epochs [1]", ylabel="Test Accuracy [%]")
ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2), ncol=3)

In [ ]:
ax = sns.scatterplot(df_model_performance, x="Runtime", y="TestAccuracy", hue="Model")
ax.set_title("Test Accuracy vs Runtime")
ax.set(xlabel="Runtime [s]", ylabel="Test Accuracy [%]")
ax.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2), ncol=3)
plt.savefig("../images/AccuracyPerSecond.png", dpi=600, bbox_inches="tight")